# Computer Lab 1 - Yanis Aït El Cadi

### Import libraries and load data

In [18]:
import numpy as np
import matplotlib.pyplot as plt

# Loading data
data_matrix_train, COP_train, data_matrix_test, COP_test, names = np.load('C:/Users/boite/Desktop/2A/SD/SD-TSIA211/TP1/data_center_data_matrix.npy', allow_pickle=True)

# Constructing matrices for min_w ||A w - b||_2**2
matrix_mean = np.mean(data_matrix_train, axis=0)
M = data_matrix_train - matrix_mean
matrix_std = np.std(M, axis=0)
M = M / matrix_std

A = np.hstack([M, np.ones((M.shape[0],1)), -(M.T * COP_train[:,3]).T])
b = COP_train[:,3]

# Constructing matrices for the test set
M_test = (data_matrix_test - matrix_mean) / matrix_std
A_test = np.hstack([M_test, np.ones((M_test.shape[0],1)), -(M_test.T * COP_test[:,3]).T])
b_test = COP_test[:,3]


# Loading raw data
import pandas as pd
data = pd.read_csv('C:/Users/boite/Desktop/2A/SD/SD-TSIA211/TP1/Raw_Dataset_May.csv')

def name_to_subcategory_and_details(col_name):
    if np.isreal(col_name):
        col_name = names[col_name]
    indices = np.nonzero((data['NAME'] == col_name).values)[0]
    if len(indices) > 0:
        subcategory = data['SUBCATEGORY'].iloc[[indices[0]]].values[0]
        details = data['DETAILS'].iloc[[indices[0]]].values[0]
        return subcategory, details
    else:
        print('unknown name')

### Question 3.1

$$
if \space Aw = b \space then \space y(t) = b_t = (Aw)_t = \tilde{x(t)}^T w_1 + w_0 - y(t) \tilde{x(t)}^T w_2 \newline
y(t)(\tilde{x(t)}^T w_2 + 1) =  \tilde{x(t)}^T w_1 + w_0 \newline
y(t) = \frac {\tilde{x(t)}^T w_1 + w_0} {\tilde{x(t)}^T w_2 + 1}
$$

### Question 3.2

In [74]:
W = np.linalg.lstsq(A,b)[0]

C:\Users\boite\AppData\Local\Temp\ipykernel_17012\299647012.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  W = np.linalg.lstsq(A,b)[0]


### Question 3.3

In [75]:
MSE = np.linalg.norm(np.dot(A_test,W)-b_test)/len(A_test)
print("The mean squared error is equal to",round(MSE,2))

The mean squared error is equal to 1.47


### Question 3.4

In [76]:
# L2 regularization strength
lambda_value = 100

# Augment the input matrix with the regularization term
A_reg = np.concatenate([A, np.sqrt(lambda_value) * np.eye(A.shape[1])], axis=0)

# Augment the target values with zeros
b_reg = np.concatenate([b, np.zeros(A.shape[1])])

# Get W
W_l2 = np.linalg.lstsq(A_reg, b_reg, rcond=None)[0]

MSE_l2 = np.linalg.norm(np.dot(A_test,W_l2)-b_test)/len(A_test)
print("The mean squared error after regularization is equal to",round(MSE_l2,2))

if MSE_l2 < MSE :
    print("MSE after regularization is better comapred to the unregularized one ! (",round(MSE_l2,2),"<",round(MSE,2),")")

The mean squared error after regularization is equal to 0.91
MSE after regularization is better comapred to the unregularized one ! ( 0.91 < 1.47 )


### Question 3.5

$ \nabla f_1(w) = A^T (Aw - b) + \lambda w $

### Question 3.6

$$
\nabla f(x) - \nabla f(y) = (A^TA+100)(x-y) \newline
|| \nabla f(x) - \nabla f(y) || <= ||(A^TA+100)||||(x-y)|| \newline
Donc \space \gamma <= \frac 2 {||A^TA+100||}
$$

In [133]:
from copy import copy, deepcopy

def f1(w):
    return 1/2 * np.linalg.norm(np.dot(A,w)-b)**2 + 50*np.linalg.norm(w)**2

def compute_gradient(w):
    error_term = np.dot(A, w) - b
    gradient = np.dot(A.T, error_term) + 100 * w
    return gradient

def gradient_descent(initial_matrix, step_size):

    current_matrix = deepcopy(initial_matrix)
    nb_iterations = 0

    while np.linalg.norm(compute_gradient(current_matrix)) > 1:

        # Compute the gradient of the matrix function
        grad = compute_gradient(current_matrix)

        # Update the matrix in the opposite direction of the gradient
        current_matrix -= step_size * grad

        nb_iterations += 1

        # print(np.linalg.norm(compute_gradient(current_matrix)))
        # print(grad)

    return current_matrix,nb_iterations

lipschitz_constant = np.linalg.norm(np.dot(A.T,A) + 100)
step_size = 2/lipschitz_constant

W_grad_desc,n_iter = gradient_descent(W, step_size)

print(n_iter,"iterations")

MSE_grad_desc = np.linalg.norm(np.dot(A_test,W_grad_desc)-b_test)/len(A_test)
print("The mean squared error with the gradient descent is equal to",round(MSE_grad_desc,2))

91164 iterations
The mean squared error with the gradient descent is equal to 0.92


We get a MSE really close to the one using the np.linalg.lstsq (0.92 compared to 0.92)

### Question 3.7

In [135]:
# def conjugate_gradient_method(initial_matrix,tolerance=1e-6,max_iter=100):
#     x = initial_matrix
#     r = b - np.dot(A, x)
#     p = r
#     rsold = np.dot(r, r)

#     for i in range(max_iter):
#         Ap = np.dot(A, p)
#         alpha = rsold / np.dot(p, Ap)
#         x = x + alpha * p
#         r = r - alpha * Ap
#         rsnew = np.dot(r, r)

#         if np.sqrt(rsnew) < tolerance:
#             break

#         beta = rsnew / rsold
#         p = r + beta * p
#         rsold = rsnew

#     return x

# W_conj_grad = conjugate_gradient_method(W)

### Question 4.1

$$
f_2(w) = \frac 1 2 ||Aw-b||^2 \space \space\space \space \nabla f_2 (w) = A^T(Aw-b) \newline
g_2(w) = \frac \lambda 2 ||w||_1 \space and \space prox_{g_2}(w) = argmin_u( g_2(u) + \frac 1 2 ||w-u||^2)
$$

### Question 4.2

In [144]:
from scipy.optimize import minimize, rosen, rosen_der

def g2(w):
    return 50 * np.max(np.abs(w))

def prox_1Lg2(w):
    def firstargmin(u):
        return 1/lipschitz_constant * g2(w) + 1/2*np.linalg.norm(w-u)**2
    return minimize(firstargmin,W)

def proximal_gradient_method(x0, max_iter=100, tol=1e-6):
    x = x0.copy()

    for iteration in range(max_iter):
        
        # Compute the gradient of the smooth part
        gradient = compute_gradient(x)
        
        # Update the parameter using the proximal operator
        x = prox_1Lg2(x - 1/lipschitz_constant * gradient)
        
        # Check for convergence

    return x

W_prox_grad = proximal_gradient_method(W)

TypeError: unsupported operand type(s) for *: 'float' and 'OptimizeResult'

In [145]:
compute_gradient(W)

array([-0.92782104,  8.30937124, -3.6727039 , ...,  1.98059531,
       -3.05717441, -1.18861393])

: 